In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd


In [2]:
image_size = 64
batch_size=32
###

In [3]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/home/dataset_scaled/train",
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=(image_size,image_size),
    batch_size=batch_size,
    label_mode='binary',
    color_mode='rgb',
    
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/home/dataset_scaled/train",
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=(image_size,image_size),
    batch_size=batch_size,
    label_mode='binary',
    color_mode='rgb',
   
)

Found 78466 files belonging to 2 classes.
Using 62773 files for training.
Found 78466 files belonging to 2 classes.
Using 15693 files for validation.


In [4]:
train_ds = train_ds.prefetch(buffer_size=32)
val_ds = val_ds.prefetch(buffer_size=32)

In [5]:
def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)
    # Image augmentation block
   # x = data_augmentation(inputs)
    x = inputs
    # Entry block
    x = layers.experimental.preprocessing.Rescaling(1.0 / 255)(x)
    x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    
    
    x = layers.Conv2D(64, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    
    previous_block_activation = x  # Set aside residual

    for size in [128, 256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)
        #x = layers.Dropout(0.5)(x)
        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)

In [6]:
#model = make_model()
my_input=(image_size,image_size,3)
model = make_model(input_shape=my_input, num_classes=2)
#keras.utils.plot_model(model, show_shapes=True)


epochs = 50

callbacks = [
    keras.callbacks.ModelCheckpoint("saved_last.h5"),
    keras.callbacks.CSVLogger('history_50.csv', separator=",", append=False),
]
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)

history = model.fit(
    train_ds, epochs=epochs, callbacks=callbacks, validation_data=val_ds,
)

Epoch 1/50
1446/1962 [=====================>........] - ETA: 2:51 - loss: 0.7214 - accuracy: 0.5023

KeyboardInterrupt: 